In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

In [2]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches


# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# # list_values.append('coarse')

# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# # Extract Key Words
# df['matches']=df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [4]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
strategy_version_input=705
attribute_id_input=4892
formatted_attribute = 'Maximum Operating Temperature'

In [5]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input} 
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()] 

In [10]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df)) 
df = df[df['bucket_id'].isin(buckets)]
height=df
len(df)
df_fasteners=df

16858


In [8]:
df['buckets'].explode().value_counts()[0:50] 

Fixings & Fasteners                                                16515
Electrical System                                                    342
Cables, Cords & Cord Management Replacement Parts & Accessories        1
Name: buckets, dtype: int64

In [10]:
active_id=[
261582,
261583,
261584,
261585,
261586,
261587,
261588,
26031,
26032,
26033,
26034,
26035,
26583,
26581,
26584,
26582,
26580,
402617,
'402611EZ',
40260,
402601,
402612,
40261,
402611,
402608,
402614,
40262,
402621,
402622,
40263,
402609,
402615,
261371,
261372,
261373,
261374,
261375,
261376,
261377,
261378,
261379,
261381,
261382,
26608,
26609,
26714,
26715,
26716,
26682,
26683,
26684,
26681,
266821,
266831,
26680,
26678,
266841,
266811,
26574,
26676,
26674,
26579,
26758,
26675,
26679,
26688,
26757,
26677,
26575,
26756,
266791,
266751,
266741,
26670,
266781,
26672,
266801,
26577,
26685,
26689,
26673,
26686,
266761,
26671,
266701,
26687,
26669,
266721,
26578,
266711,
26663,
26662,
26619,
26661,
26779,
26781,
26622,
26621,
26616,
26764,
26780,
26784,
26623,
26667,
26660,
26786,
26789,
26778,
26620,
26617,
26666,
261171,
261170,
26770,
26785,
26618,
26771,
26775,
26615,
26774,
26763,
26788,
26776,
26773,
26772,
26777,
26783,
26782,
261215,
26668,
26765,
26842,
26642,
26643,
261520,
261521,
261522,
261523,
261524,
261525,
261526,
261527,
26794,
402602,
26710,
26711,
26712,
26713,
26691,
26690,
26600,
26601,
26602,
26603,
26604,
26605,
402616,
261230,
261231,
261232,
261233,
261234,
261235]

In [12]:
regex_pattern=r'(?i)(.{30}degree.{20})|(.{30}(?<!caused by )(?<!cold )(?<!high pressure and )(?<!high )(?<!Maximum operating )temp(?!o)(?!ting)(?! RATING).{20})|(.{30}°F.{20})|(.{30}\s-\d+.{20})|()'
df_fasteners['matches']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern, x)) 
print(len(df_fasteners[df_fasteners['matches'].astype(str)!='[]'])) 
minimum=df_fasteners[df_fasteners['matches'].astype(str)!='[]']
minimum

367


product_id external_id  \
385    1173659437       85502   
386    1173651816      465520   
387    1173651797      465505   
443    1173638111       40475   
471    1173660654      375041   
626    1173647757      408551   
655    1173651879      465518   
656    1173651872      465710   
657    1173651862      465612   
659    1173651852      465912   
660    1173651846      465818   
661    1173651845      465524   
662    1173651844      348418   
663    1173651839      466010   
664    1173651837      465504   
666    1173651825      465514   
667    1173651824      348518   
668    1173651821      465824   
669    1173651817      348520   
670    1173651808      465822   
671    1173651805      465910   
672    1173651804      465516   
673    1173651800      465814   
674    1173651794      348509   
675    1173651786      465820   
676    1173651774      465508   
678    1173651768      465512   
680    1173651762      465808   
681    1173651761      465812   
682    1173651744      348506   
683    1173651742      348409   
684    1173651741      465816   
685    1173651740      348512   
686    1173651732      348414   
687    1173651730      348416   
689    1173651723      348510   
690    1173651721      348407   
691    1173651710      348408   
692    1173651630      345706   
693    1173651627      348612   
694    1173651616      348608   
695    1173651608      348606   
764    1173638109      404749   
1822   1173659462       85511   
1823   1173659460       85534   
1824   1173659459       85508   
1825   1173659458       85509   
1826   1173659454       85537   
1827   1173659451       85506   
1828   1173659449       85504   
1829   1173659447       85501   
1830   1173659445       85519   
1831   1173659442       85543   
1832   1173659441       85516   
1833   1173659440       85545   
1834   1173659435       85542   
1835   1173659434       85505   
1836   1173659433       85522   
1837   1173659432       85541   
1838   1173659428       85518   
2209   1173647260      356824   
2210   1173647258      356820   
2211   1173647256      356816   
2212   1173647251      356808   
2213   1173647250      356624   
2214   1173647246      356614   
2215   1173647245      356612   
2216   1173647243      356608   
2217   1173647242      356606   
2218   1173647241      356604   
2219   1173647236      356410   
2220   1173647235      356408   
2221   1173647232      356404   
2720   1173673503       92193   
2721   1173673459       92197   
2722   1173673419       92596   
2723   1173673414       92583   
2724   1173673409       92584   
2725   1173673346       91228   
2726   1173673339       91226   
2727   1173673338       91188   
2728   1173673335       91185   
2738   1173672853       92791   
2739   1173672851       92790   
2740   1173672838       92796   
2741   1173672833       92387   
2742   1173672781       92389   
2743   1173672769       92395   
2791   1173666682       50909   
2806   1173666656       50907   
2807   1173666655       50900   
2808   1173666652       50901   
2809   1173666650       50758   
2813   1173666641       50902   
2886   1173666409       50810   
2887   1173666408       50818   
2888   1173666407       50814   
3761   1173666515      502929   
3762   1173666514      502928   
3763   1173666511      502925   
3764   1173666510      502908   
3765   1173666508      502906   
3766   1173666507      502905   
3767   1173666504      502903   
3768   1173666502      502901   
3771   1173666488      502758   
4117   1173673497       92195   
4118   1173673476       92591   
4119   1173673458       92597   
4120   1173673456       92188   
4121   1173673454       92184   
4122   1173673452       92183   
4123   1173673449       92191   
4124   1173673447       92588   
4125   1173673435       92194   
4126   1173673431       92192   
4127   1173673428       92585   
4128   1173673332       91231   
4129   1173673329       91229   
4130   1173673327       91187   
4141   1173672

In [16]:
min_id=minimum['external_id']
import numpy as np
main_list = np.setdiff1d(active_id,min_id)
print(len(main_list))
main_list

163


array(['26031', '26032', '26033', '26034', '26035', '261170', '261171',
       '261215', '261230', '261231', '261232', '261233', '261234',
       '261235', '261371', '261372', '261373', '261374', '261375',
       '261376', '261377', '261378', '261379', '261381', '261382',
       '261520', '261521', '261522', '261523', '261524', '261525',
       '261526', '261527', '261582', '261583', '261584', '261585',
       '261586', '261587', '261588', '26574', '26575', '26577', '26578',
       '26579', '26580', '26581', '26582', '26583', '26584', '26600',
       '26601', '26602', '26603', '26604', '26605', '26608', '26609',
       '26615', '26616', '26617', '26618', '26619', '26620', '26621',
       '26622', '26623', '26642', '26643', '26660', '26661', '26662',
       '26663', '26666', '26667', '26668', '26669', '26670', '266701',
       '26671', '266711', '26672', '266721', '26673', '26674', '266741',
       '26675', '266751', '26676', '266761', '26677', '26678', '266781',
       '26679', '266791

In [103]:
# systems should not exceed
# Maximum operating temperature: PVC = 140°F / CPVC = 210°F
#  temperatures as high as
# Full rated performance up to 300°F
# TEMPERATURE RANGE: -65°F to +400°F. 
# Heat stable to 470°F.
# Maximum temperature 
# rated performance up to 300°F.
#  systems should not exceed 200°F 
regex_pattern=r'(?i)(-\d+°F to)|()'
minimum['matches']=minimum['long_desc'].apply(lambda x: re_extract(regex_pattern, x)) 
minimum['matches']=minimum['matches'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|°','',str(x))).apply(lambda x: re.sub(r"\[|\]|\'",'',str(x)))
minimum['matches'].explode().value_counts()
minimum[curation_col]=minimum['matches']
matches=minimum[['external_id',curation_col]]
matches=matches[matches[curation_col].astype(str)!='']
print(len(matches))
matches

95


external_id Q:minimum_operating_temperature
154     C3109X6X2                            -65 
155    C3109X2412                            -65 
156    C3109X20X8                            -65 
157    C3109X12X4                            -65 
158    C3109X16X6                            -65 
159     C3109X4X2                            -65 
161     C3109X8X2                            -65 
162    C3109X12X6                            -65 
164    C3109X3220                            -65 
166    C3109X3224                            -65 
167    C3109X16X8                            -65 
168    C3109X2016                            -65 
169     C3109X6X4                            -65 
170     C3109X8X4                            -65 
171    C3109X1612                            -65 
172    C3109X12X8                            -65 
173     C3109X8X6                            -65 
174    LF3220X164                            -65 
175    LF3220X122                            -65 
176    LF3220X168                            -65 
177    LF3220X8X2                            -65 
178    LF3220X6X2                            -65 
179    LF3220X126                            -65 
180    LF3220X124                            -65 
181    LF3220X4X2                            -65 
182    LF3220X6X4                            -65 
183    LF3220X128                            -65 
184    LF3220X8X4                            -65 
185    LF3220X8X6                            -65 
251      3222X4X2                            -65 
252     3222X12X8                            -65 
255     3220X16X6                            -65 
256     3220X12X2                            -65 
257     3220X16X4                            -65 
258     3220X16X8                            -65 
259      3220X8X2                            -65 
260    3220X16X12                            -65 
261      3220X6X2                            -65 
262      3220X4X2                            -65 
263     3220X12X6                            -65 
264     3220X12X8                            -65 
265      3220X6X4                            -65 
266      3220X8X4                            -65 
267      3220X8X6                            -65 
273      3222X8X6                            -65 
274      3222X8X4                            -65 
275      3222X8X2                            -65 
276      3222X6X4                            -65 
277      3222X6X2                            -65 
6169       349710                            -65 
6170       349708                            -65 
6184       349407                            -65 
6206       349405                            -65 
6207       349404                            -65 
6214       349410                            -65 
6215       349408                            -65 
6217       349406                            -65 
6240      4614520                            -65 
6241      4614516                            -65 
6242      4614514                            -65 
6243      4614510                            -65 
6244      4614508                            -65 
6245      4614506                            -65 
6246      4614505                            -65 
6247      4614504                            -65 
6248      4614503                            -65 
8944       261757                            -40 
8945       261754                            -40 
8946       261753                            -40 
8947       261751                            -40 
8948       261750                            -40 
11097     4614512                            -65 
11283      349416                            -65 
11460  C3109X2012                            -65 
11461  C3109X2416                            -65 
11465   3222X12X6                            -65 
11466   3222X12X4                            -65 
11467   3222X12X2                            -65 
12067      349412                            -65 
12271 

In [82]:
regex_pattern=r'(?i)(.{20}°F.{20})|()'
df_fasteners['matches']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_fasteners[df_fasteners['matches'].astype(str)!='[]']))
df_fasteners[df_fasteners['matches'].astype(str)!='[]']

345


product_id external_id  \
154    1173674095   C3109X6X2   
155    1173674081  C3109X2412   
156    1173674071  C3109X20X8   
157    1173674039  C3109X12X4   
158    1173674038  C3109X16X6   
159    1173674030   C3109X4X2   
160    1173673377       92594   
161    1173674020   C3109X8X2   
162    1173674018  C3109X12X6   
163    1173673347       91186   
164    1173674007  C3109X3220   
165    1173673346       91228   
166    1173674000  C3109X3224   
167    1173673990  C3109X16X8   
168    1173673988  C3109X2016   
169    1173673984   C3109X6X4   
170    1173673972   C3109X8X4   
171    1173673977  C3109X1612   
172    1173673975  C3109X12X8   
173    1173673970   C3109X8X6   
174    1173673180  LF3220X164   
175    1173673179  LF3220X122   
176    1173673178  LF3220X168   
177    1173673177  LF3220X8X2   
178    1173673176  LF3220X6X2   
179    1173673175  LF3220X126   
180    1173673174  LF3220X124   
181    1173673173  LF3220X4X2   
182    1173673171  LF3220X6X4   
183    1173673170  LF3220X128   
184    1173673169  LF3220X8X4   
185    1173673168  LF3220X8X6   
200    1173673343       91233   
201    1173673339       91226   
202    1173673338       91188   
204    1173673335       91185   
210    1173673332       91231   
211    1173673329       91229   
212    1173673328       91184   
217    1173673317       91183   
227    1173672854       92795   
228    1173672853       92791   
229    1173672850       92788   
230    1173672849       92787   
232    1173672842       92785   
233    1173672838       92796   
234    1173672829       92396   
237    1173672803       92783   
238    1173672801       92385   
244    1173672770       92397   
245    1173672767       92391   
247    1173672745       91373   
248    1173673510       92201   
249    1173673503       92193   
250    1173673501       92187   
251    1173672271    3222X4X2   
252    1173672270   3222X12X8   
253    1173673497       92195   
254    1173673490       92186   
255    1173672249   3220X16X6   
256    1173672248   3220X12X2   
257    1173672247   3220X16X4   
258    1173672246   3220X16X8   
259    1173672244    3220X8X2   
260    1173672243  3220X16X12   
261    1173672242    3220X6X2   
262    1173672241    3220X4X2   
263    1173672240   3220X12X6   
264    1173672239   3220X12X8   
265    1173672238    3220X6X4   
266    1173672237    3220X8X4   
267    1173672236    3220X8X6   
268    1173673483       92196   
269    1173673481       92189   
270    1173673476       92591   
271    1173673470       92590   
272    1173673463       92580   
273    1173672276    3222X8X6   
274    1173672275    3222X8X4   
275    1173672274    3222X8X2   
276    1173672273    3222X6X4   
277    1173672272    3222X6X2   
278    1173673458       92597   
279    1173673456       92188   
280    1173673454       92184   
281    1173673449       92191   
282    1173673447       92588   
303    1173673436       92592   
304    1173673435       92194   
305    1173673431       92192   
306    1173673428       92585   
307    1173673419       92596   
308    1173673414       92583   
309    1173673409       92584   
310    1173673407       92587   
556    1173666412       50808   
557    1173666411       50804   
558    1173666410       50802   
1501   1173660674      375028   
1502   1173660673      375026   
1503   1173660672      375024   
1504   1173660668      375032   
1505   1173660667      375031   
1506   1173660666      375030   
1507   1173660665      375029   
1508   1173660664      375039   
1509   1173660663      375034   
1510   1173660662      375036   
1511   1173660661      375033   
1512   1173660659      375021   
1513   1173660657      375023   
1514   1173660654      375041   
1942   1173659461       85510   
1943   1173659459       85508   
1944   1173659458       85509   
1945   1173659456       85544   
1946   1173659453       85538   
1947   1173659452       85531   
1948   1173659451       85506   
1949   1173659449       85504   
1950   1173659

In [85]:
regex_pattern=r'(?i)(.{20}(?<!Maximum operating )temp(?!o)(?!ting).{20})|()'
df_fasteners['matches']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_fasteners[df_fasteners['matches'].astype(str)!='[]']))
df_fasteners[df_fasteners['matches'].astype(str)!='[]']

229


product_id external_id  \
154    1173674095   C3109X6X2   
155    1173674081  C3109X2412   
156    1173674071  C3109X20X8   
157    1173674039  C3109X12X4   
158    1173674038  C3109X16X6   
159    1173674030   C3109X4X2   
161    1173674020   C3109X8X2   
162    1173674018  C3109X12X6   
164    1173674007  C3109X3220   
166    1173674000  C3109X3224   
167    1173673990  C3109X16X8   
168    1173673988  C3109X2016   
169    1173673984   C3109X6X4   
170    1173673972   C3109X8X4   
171    1173673977  C3109X1612   
172    1173673975  C3109X12X8   
173    1173673970   C3109X8X6   
174    1173673180  LF3220X164   
175    1173673179  LF3220X122   
176    1173673178  LF3220X168   
177    1173673177  LF3220X8X2   
178    1173673176  LF3220X6X2   
179    1173673175  LF3220X126   
180    1173673174  LF3220X124   
181    1173673173  LF3220X4X2   
182    1173673171  LF3220X6X4   
183    1173673170  LF3220X128   
184    1173673169  LF3220X8X4   
185    1173673168  LF3220X8X6   
251    1173672271    3222X4X2   
252    1173672270   3222X12X8   
255    1173672249   3220X16X6   
256    1173672248   3220X12X2   
257    1173672247   3220X16X4   
258    1173672246   3220X16X8   
259    1173672244    3220X8X2   
260    1173672243  3220X16X12   
261    1173672242    3220X6X2   
262    1173672241    3220X4X2   
263    1173672240   3220X12X6   
264    1173672239   3220X12X8   
265    1173672238    3220X6X4   
266    1173672237    3220X8X4   
267    1173672236    3220X8X6   
273    1173672276    3222X8X6   
274    1173672275    3222X8X4   
275    1173672274    3222X8X2   
276    1173672273    3222X6X4   
277    1173672272    3222X6X2   
343    1173666748      502780   
345    1173666740      502760   
349    1173666732      502788   
353    1173666724      502756   
355    1173666722      502752   
361    1173666714      502722   
365    1173666709      502724   
367    1173666706      502736   
374    1173666698      502732   
375    1173666696      502720   
379    1173666692      502728   
382    1173666689      502716   
384    1173666686      502740   
387    1173666683      502748   
388    1173666682       50909   
390    1173666680      502744   
392    1173666676       50908   
409    1173666656       50907   
410    1173666655       50900   
411    1173666653       50801   
412    1173666652       50901   
413    1173666650       50758   
416    1173666647       50800   
420    1173666642       50905   
421    1173666641       50902   
422    1173666640       50906   
425    1173666635       50904   
426    1173666634       50903   
477    1173666522       50929   
478    1173666521       50928   
479    1173666520       50927   
480    1173666517       50924   
481    1173666515      502929   
482    1173666514      502928   
483    1173666513      502927   
484    1173666511      502925   
485    1173666510      502908   
486    1173666509      502907   
487    1173666508      502906   
488    1173666507      502905   
489    1173666506      502904   
490    1173666503      502902   
491    1173666502      502901   
492    1173666501      502900   
494    1173666498      502801   
495    1173666497      502800   
501    1173666489      502759   
502    1173666488      502758   
1501   1173660674      375028   
1502   1173660673      375026   
1503   1173660672      375024   
1504   1173660668      375032   
1505   1173660667      375031   
1506   1173660666      375030   
1507   1173660665      375029   
1508   1173660664      375039   
1509   1173660663      375034   
1510   1173660662      375036   
1511   1173660661      375033   
1512   1173660659      375021   
1513   1173660657      375023   
1514   1173660654      375041   
1902   1173659569       85646   
1903   1173659568       85645   
1904   1173659567       85641   
1905   1173659566       85649   
1906   1173659565       85660   
1907   1173659564       85650   
1908   1173659563       85654   
1909   1173659562       85661   
1910   1173659561       85689   
1911   1173659

In [68]:
# negative values
regex_pattern=r'(?i)(.{0,20}\s-\d+.{0,20})|()'
df_fasteners['matches']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_fasteners[df_fasteners['matches'].astype(str)!='[]']))
df_fasteners[df_fasteners['matches'].astype(str)!='[]'][0:500]

170


product_id external_id  \
154    1173674095   C3109X6X2   
155    1173674081  C3109X2412   
156    1173674071  C3109X20X8   
157    1173674039  C3109X12X4   
158    1173674038  C3109X16X6   
159    1173674030   C3109X4X2   
161    1173674020   C3109X8X2   
162    1173674018  C3109X12X6   
164    1173674007  C3109X3220   
166    1173674000  C3109X3224   
167    1173673990  C3109X16X8   
168    1173673988  C3109X2016   
169    1173673984   C3109X6X4   
170    1173673972   C3109X8X4   
171    1173673977  C3109X1612   
172    1173673975  C3109X12X8   
173    1173673970   C3109X8X6   
174    1173673180  LF3220X164   
175    1173673179  LF3220X122   
176    1173673178  LF3220X168   
177    1173673177  LF3220X8X2   
178    1173673176  LF3220X6X2   
179    1173673175  LF3220X126   
180    1173673174  LF3220X124   
181    1173673173  LF3220X4X2   
182    1173673171  LF3220X6X4   
183    1173673170  LF3220X128   
184    1173673169  LF3220X8X4   
185    1173673168  LF3220X8X6   
251    1173672271    3222X4X2   
252    1173672270   3222X12X8   
255    1173672249   3220X16X6   
256    1173672248   3220X12X2   
257    1173672247   3220X16X4   
258    1173672246   3220X16X8   
259    1173672244    3220X8X2   
260    1173672243  3220X16X12   
261    1173672242    3220X6X2   
262    1173672241    3220X4X2   
263    1173672240   3220X12X6   
264    1173672239   3220X12X8   
265    1173672238    3220X6X4   
266    1173672237    3220X8X4   
267    1173672236    3220X8X6   
273    1173672276    3222X8X6   
274    1173672275    3222X8X4   
275    1173672274    3222X8X2   
276    1173672273    3222X6X4   
277    1173672272    3222X6X2   
6162   1173651872      465710   
6165   1173651852      465912   
6166   1173651849      466012   
6167   1173651843      465610   
6168   1173651824      348518   
6169   1173651823      349710   
6170   1173651822      349708   
6171   1173651821      465824   
6172   1173651816      465520   
6173   1173651815      348524   
6177   1173651805      465910   
6178   1173651804      465516   
6181   1173651800      465814   
6183   1173651797      465505   
6184   1173651787      349407   
6185   1173651786      465820   
6191   1173651779      465806   
6193   1173651776      465510   
6194   1173651774      465508   
6197   1173651768      465512   
6202   1173651762      465808   
6203   1173651761      465812   
6205   1173651759      348514   
6206   1173651756      349405   
6207   1173651753      349404   
6209   1173651749      348505   
6210   1173651745      465810   
6211   1173651744      348506   
6212   1173651742      348409   
6213   1173651740      348512   
6214   1173651739      349410   
6215   1173651738      349408   
6216   1173651736      348420   
6217   1173651734      349406   
6219   1173651732      348414   
6220   1173651731      348508   
6221   1173651730      348416   
6224   1173651721      348407   
6226   1173651718      348412   
6228   1173651713      348405   
6229   1173651712      348410   
6230   1173651711      348406   
6231   1173651710      348408   
6233   1173651708      465920   
6234   1173651707      465918   
6235   1173651704      465914   
6236   1173651703      465908   
6240   1173651698     4614520   
6241   1173651697     4614516   
6242   1173651696     4614514   
6243   1173651694     4614510   
6244   1173651693     4614508   
6245   1173651692     4614506   
6246   1173651691     4614505   
6247   1173651690     4614504   
6248   1173651689     4614503   
6261   1173651657      345712   
6279   1173651652      345710   
6280   1173651650      345708   
6288   1173651630      345706   
6290   1173651629      345704   
6293   1173651627      348612   
6298   1173651616      348608   
6304   1173651608      348606   
6310   1173651603      348604   
6835   1173651601      348605   
6838   1173651595      348610   
8944   1173647688      261757   
8945   1173647684      261754   
8946   1173647683      261753   
8947   1173647681      261751   
8948   1173647

# N/A Values

In [11]:
regex_pattern=r'(?i)(.{20}degree.{20})|(.{20}temp(?!o)(?!ting).{20})|(.{20}°F.{20})|(.{20}\s-\d+.{20})|()'
df_fasteners['matches']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern, x)) 
print(len(df_fasteners[df_fasteners['matches'].astype(str)=='[]'])) 
df_na=df_fasteners[df_fasteners['matches'].astype(str)=='[]']

16483


In [13]:
df_na[0:500]

product_id external_id  \
0    1173655993      337654   
1    1173655989      337666   
2    1173655986      337760   
3    1173655984      337660   
4    1173655983      337766   
5    1173655981      337466   
6    1173655978      337254   
7    1173655967      337756   
8    1173655958      337706   
9    1173655957      337303   
10   1173655956      337202   
11   1173655955      337210   
12   1173655954      337106   
13   1173655953      337405   
14   1173655951      337104   
15   1173655949      337205   
16   1173655947      337314   
17   1173655930      337304   
18   1173655929      337204   
19   1173655926      337606   
20   1173655925      337710   
21   1173655922      337716   
22   1173655921      337416   
23   1173655919      337410   
24   1173655913      337306   
25   1173655910      337785   
26   1173655904      337183   
27   1173655902      337282   
28   1173655900     3337686   
29   1173655899      337184   
30   1173655898      337186   
31   1173655897      337398   
32   1173655887     3337688   
33   1173655881     3337286   
34   1173655879     3337386   
35   1173655874      337784   
36   1173655872     3337788   
37   1173655871      337690   
38   1173655870     3337388   
39   1173655869      337686   
40   1173655867     3337492   
41   1173655862      337288   
42   1173655860      337384   
43   1173655859      337284   
44   1173655857      337788   
45   1173655853      337490   
46   1173655852      337692   
47   1173655849      337792   
48   1173655612     4310412   
49   1173655428      449060   
50   1173655424      448654   
51   1173655423      448858   
52   1173655421      449254   
53   1173655395      441612   
54   1173655388      441708   
55   1173655387      449412   
56   1173655381      441606   
57   1173655380      442312   
58   1173655379      442252   
59   1173655373      442246   
60   1173655372      442248   
61   1173655370      441912   
62   1173655369      441910   
63   1173655366      449212   
64   1173655365      442308   
65   1173655359      442046   
66   1173655356      441906   
67   1173655354      449006   
68   1173655340      449439   
69   1173655338      443805   
70   1173655335      444012   
71   1173655330      443806   
72   1173655326      430405   
73   1173655325      430406   
74   1173655320      430602   
75   1173655318      430408   
76   1173655314      430812   
77   1173655268      440706   
78   1173655262       10458   
79   1173655256      440820   
80   1173655249       10561   
81   1173655237      440605   
82   1173655234      440612   
83   1173655208       10932   
84   1173655207       10933   
85   1173655204      440803   
86   1173655198      441010   
87   1173655191      441004   
88   1173655186      442152   
89   1173655184      442138   
90   1173655159       12549   
91   1173655158       10460   
92   1173655156       10461   
93   1173655154       11200   
94   1173655141       12553   
95   1173655137       12619   
96   1173655133       10466   
97   1173655130       12531   
98   1173655129       12544   
99   1173655125       12608   
100  1173655113       10392   
101  1173655107       11903   
102  1173655104       10942   
103  1173655102       10952   
104  1173655097       11906   
105  1173655096       12524   
106  1173655095      11900A   
107  1173655094      11908A   
108  1173655087       11925   
109  1173655068       12611   
110  1173655064       12613   
111  1173655061       11918   
112  1173655059       11917   
113  1173655054       11215   
114  1173655013       12742   
115  1173654994       12953   
116  1173654990       12960   
117  1173654984       12749   
118  1173654979       10381   
119  1173654978       10382   
120  1173654970       12811   
121  1173654968       12668   
122  1173654967       12825   
123  1173654963       12822   
124  1173654959       12812   
125  1173654957      441603   
126  1173654949       12688   
127  1173654948       12830   
128  1

In [14]:
df_na[curation_col]='n/a'
matches_na=df_na[['external_id',curation_col]]
matches_na

external_id Q:maximum_operating_temperature
0          337654                             n/a
1          337666                             n/a
2          337760                             n/a
3          337660                             n/a
4          337766                             n/a
...           ...                             ...
16853      336540                             n/a
16854      432816                             n/a
16855       22275                             n/a
16856      469312                             n/a
16857      335220                             n/a

[16483 rows x 2 columns]

In [23]:
matches_na

external_id Q:maximum_operating_temperature
0          337654                             n/a
1          337666                             n/a
2          337760                             n/a
3          337660                             n/a
4          337766                             n/a
...           ...                             ...
16853      336540                             n/a
16854      432816                             n/a
16855       22275                             n/a
16856      469312                             n/a
16857      335220                             n/a

[16483 rows x 2 columns]

In [ ]:
main_list_na = np.setdiff1d(active_id,na_list)
print(len(main_list_na))
main_list_na

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [26]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Fasteners- {attribute}-- value- upload_Shawn.csv',index=False)

In [27]:
looks_good('Kimball Midwest', attribute, df, matches_na)

In [21]:
def looks_good_na(customer, attribute, null, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = null[null['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Fasteners - {attribute} - na upload.csv',index=False)
#     matches.to_csv(f'{drive_path}/Fasteners - {attribute}-- total_na_values upload_Shawn.csv',index=False)

In [22]:
looks_good_na('Kimball Midwest', attribute, df, matches_na)